In [4]:
import numpy as np
from torch import nn, optim
import torch
import random
import time

🤔思考：如何加载数据？

In [5]:
with open("labels.txt", "r") as f:
    raw = f.readlines()

tags = []
data = []
for l in raw:
    tags.append(int(l[0]))#每行的第一个字符是标签
    d = l[1:-1]#去掉标签和换行符
    d = map(float, tuple(d)) #将字符串转换为tuple，数字转换为float，方便后续转为tensor
    #tuple相对于list更省内存，因为tuple是不可变的，对象所含method更少
    data.append(tuple(d))

#将标签和数据转为tensor，方便后续切分训练集和测试集
data = torch.tensor(data)
tags = torch.tensor(tags)

#划分训练集和测试集
train_test_ratio = 0.8
train_size = int(train_test_ratio * len(data))
test_size = len(data) - train_size
data_train = data[:train_size]
data_test  = data[train_size:]
tags_train = tags[:train_size]
tags_test  = tags[train_size:]

In [6]:
# 直接套用d2l网站上的代码，没有改动
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


batch_size = 10

w = torch.normal(0, 0.01, size=(len(data[0]), 1), requires_grad=True, dtype=torch.float32) #对每个像素都有一个权重
b = torch.zeros(1, requires_grad=True)

👇方案3：Multi-Layer Perceptron (MLP)